In [1]:
import numpy as np

# Back-propagation (Práctica)

Entrenar un aproximador para la función XOR usando 2 capas intermedias.
* Usar 2 neuronas en la capa anterior a la salida (segunda capa oculta=
* Usar al menos 2( pueden ser más) en la primera capa oculta.
* Usar activación ReLU en las capas intermedias y no activación en la salida

Usar Numpy.

Realizar 5 experimentos, en cada experimento (corrida de entrenamiento):
* Inicializar los parámetros aleatoriamente con distribución normal centrada en 0 y std = 0.1
* Retornar la representación intermedia de la segunda capa oculta.

Graficar las 5 representaciones intermedias, comparar, comentar y/o concluir.


In [303]:
def relu(X):
    return np.maximum(0,X) 


def sigmoid(X):
    return 1/(1+np.exp(-X))

def derivative_relu(array):
    return np.array([n > 0 for n in array], dtype = np.int)

In [627]:
class NeuralNetXOR:
    def __init__(self, X, y_real):
        self.X = X
        self.y_real = y_real
        self.w_1 = np.random.normal(0, 0.1, (2,3))
        self.w_2 = np.random.normal(0, 0.1, (2,3))
        self.w_3 = np.random.normal(0, 0.1, (1,3))
        
    def forward_propagation(self):
        X_input = np.append(np.ones((self.X.shape[0], 1)), self.X, axis = -1)
        l1 = np.matmul(X_input, self.w_1.T)
        l1_a = relu(l1)
        l1_a_w_bias = np.append(np.ones((l1_a.shape[0], 1)),l1_a, axis = -1)
        l2 = np.matmul(l1_a_w_bias, self.w_2.T)
        l2_a = relu(l2)
        l2_a_w_bias = np.append(np.ones((l2_a.shape[0], 1)), l2_a, axis = -1 )
        output = np.matmul(l2_a_w_bias, self.w_3.T)
        output_a = sigmoid(output)
        
        return l1, l1_a, l2, l2_a, output, output_a
    
    def error(self):
        _,_,_,_,_,output = self.forward_propagation()
        
        cross_entropy = np.mean(y_real * (-np.log(output)) + (1 - y_real) * (-np.log(1 - output)))
        
        return cross_entropy
    
    def back_prop(self):
        error = self.error()
        l1, l1_a, l2, l2_a, output, output_a = self.forward_propagation()
        d_error_d_output = (output_a - y_real)
        d_error_d_w_3 = np.matmul(l2_a.T,d_error_d_output)
        d_error_d_l2 =  np.matmul(d_error_d_output, self.w_3[:,1:3]) * derivative_relu(l2)
        d_error_d_w_2 = np.matmul(l1_a.T,d_error_d_l2)
        d_error_d_l1 = np.matmul(d_error_d_l2,self.w_2[:,1:3]) * derivative_relu(l1)
        d_error_d_w_1 = np.matmul(self.X.T,d_error_d_l1)
        d_error_b_3 = np.sum(d_error_d_output, axis = 0, keepdims =True)
        d_error_b_2 = np.sum(d_error_d_l2, axis = 0, keepdims = True)
        d_error_b_1 = np.sum(d_error_d_l1, axis = 0, keepdims = True)        
        
        return d_error_d_w_3, d_error_d_w_2, d_error_d_w_1, d_error_b_3, d_error_b_2, d_error_b_1
    
    def fit(self, lr, epochs):
        for i in range(0, epochs):
            d_w_3, d_w_2, d_w_1, d_b_3, d_b_2, d_b_1 = self.back_prop()
            self.w_3[:,:1] = self.w_3[:,:1][0] - lr * d_b_3[0]
            self.w_3[:,1:3] = self.w_3[:,1:3][0] - lr * d_w_3[0]
            self.w_2[:,:1] = self.w_2[:,:1][:,0] - lr * d_b_2[:,0]
            self.w_2[:,1:3] = self.w_2[:,1:3][:,0] - lr * d_w_2[:,0]
            self.w_1[:,:1] = self.w_1[:,:1][:,0] - lr * d_b_2[:,0]
            self.w_1[:,1:3] = self.w_1[:,1:3][:,0] - lr * d_w_3[:,0]
            
            print("Epoch: " + str(i) + str(self.error()))
    
        
        

In [628]:
X_inputs = np.array([[0,0], [0,1], [1,0], [1,1]])

In [629]:
y_real = np.array([[0], [1], [1], [0]])

In [630]:
nn = NeuralNetXOR(X_inputs, y_real)

In [631]:
nn.back_prop()[4] * 0.01

array([[0., 0.]])

In [632]:
nn.w_3[:,1:3][0] - nn.back_prop()[0][0] * 0.01

array([ 0.01796828, -0.00791129])

In [633]:
np.subtract(nn.w_3[:,1:3][0],0.01 * nn.back_prop()[0][0])

array([ 0.01796828, -0.00791129])

In [634]:
 - 0.01 * nn.back_prop()[4]

array([[-0., -0.]])

In [635]:
nn.w_2[:,:1][:,0] - 0.01 * nn.back_prop()[4][0]

array([-0.02677995, -0.10159127])

In [636]:
nn.fit(0.01,50)

ValueError: could not broadcast input array from shape (2) into shape (2,1)